In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
data_original = pd.read_csv('../in/datos_originales_empresa.csv')
data_original.rename(columns={'material':'unique_id', 'fecha_semana':'ds', 'cantidad':'y'}, inplace=True)
data_original.set_index('ds', inplace=True)
data_original.index = pd.to_datetime(data_original.index)
data_original.sort_values(by=['unique_id', 'ds'], inplace=True)
data_original.head(5)

,unique_id,y
ds,,
2015-12-21,000010,3.0
2016-01-11,000010,5.0
2016-01-18,000010,3.0
2016-02-22,000010,9.0
2016-03-07,000010,1.0


## Data Preprocessing

#### 1. Complete dates & Impute zeros

In [4]:
df_complete_date_range = pd.DataFrame()

for product in data_original.unique_id.unique():
    df_one_id = data_original[data_original['unique_id'] == product]
    new_dates = pd.date_range(df_one_id.index.min().date(), df_one_id.index.max().date(), freq='7D')
    df_one_id = df_one_id.reindex(new_dates)
    df_one_id['unique_id'] = product
    df_one_id = df_one_id.fillna(0)
    df_complete_date_range = pd.concat([df_complete_date_range, df_one_id])

df_complete_date_range.head()


,unique_id,y
2015-12-21,000010,3.0
2015-12-28,000010,0.0
2016-01-04,000010,0.0
2016-01-11,000010,5.0
2016-01-18,000010,3.0


In [5]:
def cumsum_restart_zero(column):
    cumsum = 0
    cumsum_list = []
    for value in column:
        if value == 0:
            cumsum = 0
        cumsum += value
        cumsum_list.append(cumsum)
    return cumsum_list

In [6]:
df_complete_date_range['week_with_sale'] = np.where(df_complete_date_range['y'] > 0, 1, 0)
df_complete_date_range['cumulative_weeks'] = df_complete_date_range.groupby('unique_id')['week_with_sale'].transform(cumsum_restart_zero)
df_complete_date_range

,unique_id,y,week_with_sale,cumulative_weeks
2015-12-21,000010,3.0,1,1
2015-12-28,000010,0.0,0,0
2016-01-04,000010,0.0,0,0
2016-01-11,000010,5.0,1,1
2016-01-18,000010,3.0,1,2
...,...,...,...,...
2016-10-31,Z194161060,0.0,0,0
2016-11-07,Z194161060,0.0,0,0
2016-11-14,Z194161060,0.0,0,0
2016-11-21,Z194161060,0.0,0,0


#### 2. Exclude test period

In [ ]:
df_without_test_period = pd.DataFrame()

for product in df_complete_date_range.unique_id.unique():
    df_one_id = df_complete_date_range[df_complete_date_range['unique_id'] == product]
    index_9w = df_one_id.index[df_one_id['cumulative_weeks']==9].min()
    index_1w = index_9w - timedelta(weeks=8)
    df_no_test = df_one_id[df_one_id.index >= index_1w]
    df_without_test_period = pd.concat([df_without_test_period, df_no_test])

df_without_test_period

#### 3. Products selection

We load a pre-computed CSV file because the execution of certain steps takes several minutes to complete.

In [8]:
#df_without_test_period.to_csv('../out/id_selection/df_without_test_period.csv')
df_without_test_period = pd.read_csv('../out/id_selection/df_without_test_period.csv')
df_without_test_period.rename(columns={'Unnamed: 0':'ds'}, inplace=True)
df_without_test_period.set_index('ds', inplace=True)
df_without_test_period.index = pd.to_datetime(df_without_test_period.index)
df_without_test_period

,unique_id,y,week_with_sale,cumulative_weeks
ds,,,,
2016-12-05,001010,8.0,1,1
2016-12-12,001010,13.0,1,2
2016-12-19,001010,5.0,1,3
2016-12-26,001010,1.0,1,4
2017-01-02,001010,14.0,1,5
...,...,...,...,...
2023-01-30,Y71144701,0.0,0,0
2023-02-06,Y71144701,28.0,1,1
2023-02-13,Y71144701,106.0,1,2


In [9]:
t = df_without_test_period.index.max()
t2 = t - timedelta(weeks= 8)

df_without_test_period.reset_index(inplace=True)

df_summary = pd.DataFrame()
df_summary['min_date'] = df_without_test_period.groupby('unique_id')['ds'].min()
df_summary['max_date'] = df_without_test_period.groupby('unique_id')['ds'].max()
df_summary['lifetime'] = (df_summary['max_date'] - df_summary['min_date']).dt.days /7
df_summary['lifetime'] = df_summary['lifetime'].round()
df_summary['n_records'] = df_without_test_period.groupby('unique_id')['y'].count()
df_summary['n_zero'] = df_without_test_period[df_without_test_period['y'] == 0].groupby('unique_id')['y'].count()
df_summary['n_zero'] = df_summary['n_zero'].fillna(0)
df_summary['% zero'] = ((df_summary['n_zero']/df_summary['n_records'])*100).round(1)
df_summary['n_consecutive_weeks'] = df_without_test_period.groupby('unique_id')['cumulative_weeks'].max()
df_summary['ok_zeros'] = np.where(df_summary['% zero'] < 20,1, 0)
df_summary['ok_2years'] = np.where(df_summary['min_date'] < '2021-01-01', 1, 0)
df_summary['ok_sales_last_2months'] = np.where(df_summary['max_date'] > t2,1, 0)
df_summary['time_series'] = np.where((df_summary['ok_zeros'] == 1) &(df_summary['ok_2years'] == 1) & (df_summary['ok_sales_last_2months'] == 1) , 1, 0)

df_summary

,min_date,max_date,lifetime,n_records,n_zero,% zero,n_consecutive_weeks,ok_zeros,ok_2years,ok_sales_last_2months,time_series
unique_id,,,,,,,,,,,
001010,2016-12-05,2023-02-27,325.0,326,102.0,31.3,14,0,1,1,0
002010,2017-01-02,2023-02-13,319.0,320,148.0,46.2,13,0,1,1,0
00610101,2015-12-14,2023-03-06,377.0,378,0.0,0.0,378,1,1,1,1
00610101AC,2020-05-04,2023-02-27,147.0,148,12.0,8.1,29,1,1,1,1
00610101ACD,2016-03-14,2022-01-31,307.0,308,150.0,48.7,18,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
T194161520,2022-08-29,2023-01-09,19.0,20,2.0,10.0,17,1,0,0,0
Y71110301,2022-05-02,2023-02-27,43.0,44,8.0,18.2,28,1,0,1,0
Y71126701,2022-01-03,2023-02-27,60.0,61,12.0,19.7,23,1,0,1,0


In [10]:
id_selected = df_summary[df_summary['time_series']==1]
len(id_selected.index.unique())

604

In [6]:
#id_selected.to_csv('../out/id_selection/id_selected.csv')
#df_summary.to_csv('../out/id_selection/ids_summary_complete.csv')

In [11]:
df_without_test_period = df_without_test_period[df_without_test_period['unique_id'].isin(id_selected.index.unique())]

#### 4. Replace Outliers

In [12]:
def replace_outliers(df, target):
    q3 = np.quantile(df[target], 0.75)
    q1 = np.quantile(df[target], 0.25)
    iqr = q3- q1
    max_limit = q3 + (1.5 * iqr)
    min_limit = q1 - (1.5 * iqr)
    df.loc[df[target] > max_limit, target] = max_limit
    df.loc[df[target] < min_limit, target] = min_limit

    return df

In [13]:
df_without_outliers = pd.DataFrame() 
for product in df_without_test_period.unique_id.unique():
    df_one_id = df_without_test_period[df_without_test_period['unique_id'] == product]
    df_one_id = replace_outliers(df_one_id, 'y')
    df_without_outliers = pd.concat([df_without_outliers, df_one_id])

df_without_outliers

,ds,unique_id,y,week_with_sale,cumulative_weeks
646,2015-12-14,00610101,10.0,1,1
647,2015-12-21,00610101,47.0,1,2
648,2015-12-28,00610101,118.0,1,3
649,2016-01-04,00610101,227.0,1,4
650,2016-01-11,00610101,220.0,1,5
...,...,...,...,...,...
472637,2023-01-30,999777,29.0,1,13
472638,2023-02-06,999777,28.0,1,14
472639,2023-02-13,999777,29.0,1,15
472640,2023-02-20,999777,17.0,1,16


In [14]:
df_without_outliers.reset_index(inplace=True)
df_without_outliers.set_index('unique_id', inplace=True)
df_without_outliers = df_without_outliers[['ds','y']]
df_without_outliers

,ds,y
unique_id,,
00610101,2015-12-14,10.0
00610101,2015-12-21,47.0
00610101,2015-12-28,118.0
00610101,2016-01-04,227.0
00610101,2016-01-11,220.0
...,...,...
999777,2023-01-30,29.0
999777,2023-02-06,28.0
999777,2023-02-13,29.0


#### 5. Adjust Series' Length

In [15]:
df_without_outliers = df_without_outliers[df_without_outliers['ds']>='2019-01-01']
df_without_outliers = df_without_outliers[df_without_outliers['ds']<'2023-01-01']
#df_without_outliers.to_csv('../out/sales_files/weekly_sales_selected_loop_without_test_outliers_2019-2022.csv')
df_without_outliers

,ds,y
unique_id,,
00610101,2019-01-07,97.0
00610101,2019-01-14,95.0
00610101,2019-01-21,124.0
00610101,2019-01-28,143.0
00610101,2019-02-04,216.0
...,...,...
999777,2022-11-28,29.0
999777,2022-12-05,29.0
999777,2022-12-12,29.0


* Log transformation is included in the modeling file for practicality.